In [1]:
import mlflow 
import warnings, git, hashlib, os
from prefect import flow, task 

In [2]:
#Github config

try:
    repo = git.Repo(search_parent_directories=True)
    sha = repo.head.commit.hexsha
except:
    warnings.warn('No github repo - generating new SHA')
    sha = hashlib.algorithms_guaranteed

In [3]:
#Mlflow config

name = 'Best Classifier'
experiment = mlflow.get_experiment_by_name(name)
if not experiment:
    experiment_id=mlflow.create_experiment(name)
experiment = mlflow.set_experiment(name)
run_params = {"experiment_id" : experiment.experiment_id,
              "description" : "Testando diferentes classificadores para dados artificiais com prefect e mlflow",
              "tags" : {'release.version':'0.0.1'}}
print("Experiment_id: {}".format(experiment.experiment_id))
print("Artifact Location: {}".format(experiment.artifact_location))
print("Tags: {}".format(experiment.tags))
print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))

Experiment_id: 1
Artifact Location: file:///home/lpfgarcia/Projects/MLOps/practical-prefect/mlruns/1
Tags: {}
Lifecycle_stage: active


In [4]:
from sklearn.datasets import make_classification

@task
def generate_data(n_samples, n_features):
    return make_classification(n_samples, n_features, n_informative=2, n_redundant=2, random_state=42)

In [5]:
from sklearn.model_selection import train_test_split

@task
def split_data(X, y):
    return train_test_split(X, y, shuffle=False, test_size=int(len(X)*0.1))

In [6]:
@flow
def generate_split(n_samples, n_features):
    X, y = generate_data(n_samples, n_features)
    return split_data(X, y)

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

def classifiers():
    
    clf_list = [
        (KNeighborsClassifier(3), "Nearest Neighbors"),
        (SVC(kernel="linear", C=0.025), "Linear SVM"),
        (DecisionTreeClassifier(max_depth=5), "Decision Tree"),
        (RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1), "Random Forest"),
        (AdaBoostClassifier(), "AdaBoost"),
        (GaussianNB(), "Naive Bayes"),
    ]
    
    return clf_list

In [8]:
@task
def fit(clf, X_train, y_train):
    return clf.fit(X_train, y_train)

In [9]:
@task
def predict(model, X_test):
    return model.predict(X_test)

In [10]:
from sklearn.metrics import accuracy_score

@task
def performance(y_test, y_pred):
    return accuracy_score(y_test, y_pred)

In [11]:
@flow
def evaluate_classifier(clf, X_train, X_test, y_train, y_test):
    model = fit(clf, X_train, y_train)
    y_pred = predict(model, X_test)
    return model, y_pred, performance(y_test, y_pred)

In [12]:
@flow(version=sha)
def evaluate(samples, features):
    """Avaliação de diferentes classificadores para cada combinação de parâmetro da base de dados"""
    clf_list = classifiers()
    X_train, X_test, y_train, y_test = generate_split(samples, features)

    for clf, name in clf_list:
    
        with mlflow.start_run() as run:

            mlflow.log_param('name', name)
            mlflow.log_param('samples', samples)  
            mlflow.log_param('features', features)  

            model, y_pred, acc = evaluate_classifier(clf, X_train, X_test, y_train, y_test)

            mlflow.sklearn.log_model(
                sk_model = model,
                artifact_path = name
            )

            mlflow.log_metric('accuracy', acc)

In [13]:
samples = range(10000, 50000, 10000)
features = range(20, 30, 2)

combination = [(x,y) for x in samples for y in features]
    
for samples, features in combination:
    evaluate(samples, features)

13:27:58.544 | INFO    | prefect.engine - Created flow run 'clever-catfish' for flow 'evaluate'
13:27:58.731 | INFO    | Flow run 'clever-catfish' - Created subflow run 'crazy-ostrich' for flow 'generate-split'
13:27:58.789 | INFO    | Flow run 'crazy-ostrich' - Created task run 'generate_data-eeb4694a-0' for task 'generate_data'
13:27:58.790 | INFO    | Flow run 'crazy-ostrich' - Executing 'generate_data-eeb4694a-0' immediately...
13:27:58.865 | INFO    | Task run 'generate_data-eeb4694a-0' - Finished in state Completed()
13:27:58.888 | INFO    | Flow run 'crazy-ostrich' - Created task run 'split_data-b2f518fa-0' for task 'split_data'
13:27:58.888 | INFO    | Flow run 'crazy-ostrich' - Executing 'split_data-b2f518fa-0' immediately...
13:27:58.955 | INFO    | Task run 'split_data-b2f518fa-0' - Finished in state Completed()
13:27:58.999 | INFO    | Flow run 'crazy-ostrich' - Finished in state Completed()
13:27:59.095 | INFO    | Flow run 'clever-catfish' - Created subflow run 'friendly-

13:28:07.996 | INFO    | Task run 'split_data-b2f518fa-0' - Finished in state Completed()
13:28:08.045 | INFO    | Flow run 'dark-toad' - Finished in state Completed()
13:28:08.114 | INFO    | Flow run 'uncovered-magpie' - Created subflow run 'mottled-angelfish' for flow 'evaluate-classifier'
13:28:08.169 | INFO    | Flow run 'mottled-angelfish' - Created task run 'fit-2157b1fd-0' for task 'fit'
13:28:08.170 | INFO    | Flow run 'mottled-angelfish' - Executing 'fit-2157b1fd-0' immediately...
13:28:08.237 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
13:28:08.260 | INFO    | Flow run 'mottled-angelfish' - Created task run 'predict-f915ca64-0' for task 'predict'
13:28:08.260 | INFO    | Flow run 'mottled-angelfish' - Executing 'predict-f915ca64-0' immediately...
13:28:08.424 | INFO    | Task run 'predict-f915ca64-0' - Finished in state Completed()
13:28:08.443 | INFO    | Flow run 'mottled-angelfish' - Created task run 'performance-0da4127c-0' for task 'performanc

13:28:17.188 | INFO    | Flow run 'cherubic-numbat' - Executing 'predict-f915ca64-0' immediately...
13:28:17.346 | INFO    | Task run 'predict-f915ca64-0' - Finished in state Completed()
13:28:17.366 | INFO    | Flow run 'cherubic-numbat' - Created task run 'performance-0da4127c-0' for task 'performance'
13:28:17.366 | INFO    | Flow run 'cherubic-numbat' - Executing 'performance-0da4127c-0' immediately...
13:28:17.411 | INFO    | Task run 'performance-0da4127c-0' - Finished in state Completed()
13:28:17.460 | INFO    | Flow run 'cherubic-numbat' - Finished in state Completed()
13:28:18.249 | INFO    | Flow run 'serious-hippo' - Created subflow run 'charcoal-oyster' for flow 'evaluate-classifier'
13:28:18.305 | INFO    | Flow run 'charcoal-oyster' - Created task run 'fit-2157b1fd-0' for task 'fit'
13:28:18.306 | INFO    | Flow run 'charcoal-oyster' - Executing 'fit-2157b1fd-0' immediately...
13:28:18.882 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
13:28:18.913

13:28:27.255 | INFO    | Flow run 'hot-shellfish' - Created subflow run 'ruddy-clam' for flow 'evaluate-classifier'
13:28:27.304 | INFO    | Flow run 'ruddy-clam' - Created task run 'fit-2157b1fd-0' for task 'fit'
13:28:27.305 | INFO    | Flow run 'ruddy-clam' - Executing 'fit-2157b1fd-0' immediately...
13:28:27.923 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
13:28:27.946 | INFO    | Flow run 'ruddy-clam' - Created task run 'predict-f915ca64-0' for task 'predict'
13:28:27.947 | INFO    | Flow run 'ruddy-clam' - Executing 'predict-f915ca64-0' immediately...
13:28:28.074 | INFO    | Task run 'predict-f915ca64-0' - Finished in state Completed()
13:28:28.098 | INFO    | Flow run 'ruddy-clam' - Created task run 'performance-0da4127c-0' for task 'performance'
13:28:28.098 | INFO    | Flow run 'ruddy-clam' - Executing 'performance-0da4127c-0' immediately...
13:28:28.148 | INFO    | Task run 'performance-0da4127c-0' - Finished in state Completed()
13:28:28.181 | INFO 

13:28:37.199 | INFO    | Task run 'predict-f915ca64-0' - Finished in state Completed()
13:28:37.225 | INFO    | Flow run 'papaya-caterpillar' - Created task run 'performance-0da4127c-0' for task 'performance'
13:28:37.225 | INFO    | Flow run 'papaya-caterpillar' - Executing 'performance-0da4127c-0' immediately...
13:28:37.284 | INFO    | Task run 'performance-0da4127c-0' - Finished in state Completed()
13:28:37.315 | INFO    | Flow run 'papaya-caterpillar' - Finished in state Completed()
13:28:38.097 | INFO    | Flow run 'classy-quoll' - Created subflow run 'phenomenal-sawfish' for flow 'evaluate-classifier'
13:28:38.154 | INFO    | Flow run 'phenomenal-sawfish' - Created task run 'fit-2157b1fd-0' for task 'fit'
13:28:38.155 | INFO    | Flow run 'phenomenal-sawfish' - Executing 'fit-2157b1fd-0' immediately...
13:28:38.298 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
13:28:38.321 | INFO    | Flow run 'phenomenal-sawfish' - Created task run 'predict-f915ca64-0' 

...

13:31:12.525 | INFO    | Flow run 'incredible-quokka' - Executing 'fit-2157b1fd-0' immediately...
13:31:12.629 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
13:31:12.651 | INFO    | Flow run 'incredible-quokka' - Created task run 'predict-f915ca64-0' for task 'predict'
13:31:12.652 | INFO    | Flow run 'incredible-quokka' - Executing 'predict-f915ca64-0' immediately...
13:31:12.907 | INFO    | Task run 'predict-f915ca64-0' - Finished in state Completed()
13:31:12.930 | INFO    | Flow run 'incredible-quokka' - Created task run 'performance-0da4127c-0' for task 'performance'
13:31:12.930 | INFO    | Flow run 'incredible-quokka' - Executing 'performance-0da4127c-0' immediately...
13:31:12.982 | INFO    | Task run 'performance-0da4127c-0' - Finished in state Completed()
13:31:13.061 | INFO    | Flow run 'incredible-quokka' - Finished in state Completed()
13:31:13.840 | INFO    | Flow run 'armored-narwhal' - Created subflow run 'stoic-python' for flow 'evaluate-class

13:31:33.046 | INFO    | Flow run 'charcoal-coati' - Executing 'performance-0da4127c-0' immediately...
13:31:33.097 | INFO    | Task run 'performance-0da4127c-0' - Finished in state Completed()
13:31:33.187 | INFO    | Flow run 'charcoal-coati' - Finished in state Completed()
13:31:34.001 | INFO    | Flow run 'turquoise-orangutan' - Created subflow run 'garnet-peccary' for flow 'evaluate-classifier'
13:31:34.059 | INFO    | Flow run 'garnet-peccary' - Created task run 'fit-2157b1fd-0' for task 'fit'
13:31:34.059 | INFO    | Flow run 'garnet-peccary' - Executing 'fit-2157b1fd-0' immediately...
13:31:42.157 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
13:31:42.180 | INFO    | Flow run 'garnet-peccary' - Created task run 'predict-f915ca64-0' for task 'predict'
13:31:42.181 | INFO    | Flow run 'garnet-peccary' - Executing 'predict-f915ca64-0' immediately...
13:31:42.741 | INFO    | Task run 'predict-f915ca64-0' - Finished in state Completed()
13:31:42.765 | INFO  

13:31:54.851 | INFO    | Flow run 'nondescript-dragon' - Executing 'fit-2157b1fd-0' immediately...
13:32:03.379 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
13:32:03.400 | INFO    | Flow run 'nondescript-dragon' - Created task run 'predict-f915ca64-0' for task 'predict'
13:32:03.401 | INFO    | Flow run 'nondescript-dragon' - Executing 'predict-f915ca64-0' immediately...
13:32:03.996 | INFO    | Task run 'predict-f915ca64-0' - Finished in state Completed()
13:32:04.015 | INFO    | Flow run 'nondescript-dragon' - Created task run 'performance-0da4127c-0' for task 'performance'
13:32:04.016 | INFO    | Flow run 'nondescript-dragon' - Executing 'performance-0da4127c-0' immediately...
13:32:04.068 | INFO    | Task run 'performance-0da4127c-0' - Finished in state Completed()
13:32:04.112 | INFO    | Flow run 'nondescript-dragon' - Finished in state Completed()
13:32:04.945 | INFO    | Flow run 'silver-skunk' - Created subflow run 'energetic-civet' for flow 'evaluate

13:32:26.106 | INFO    | Task run 'performance-0da4127c-0' - Finished in state Completed()
13:32:26.153 | INFO    | Flow run 'natural-puma' - Finished in state Completed()
13:32:26.943 | INFO    | Flow run 'ivory-buzzard' - Created subflow run 'furry-caiman' for flow 'evaluate-classifier'
13:32:26.999 | INFO    | Flow run 'furry-caiman' - Created task run 'fit-2157b1fd-0' for task 'fit'
13:32:27.000 | INFO    | Flow run 'furry-caiman' - Executing 'fit-2157b1fd-0' immediately...
13:32:27.474 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
13:32:27.498 | INFO    | Flow run 'furry-caiman' - Created task run 'predict-f915ca64-0' for task 'predict'
13:32:27.498 | INFO    | Flow run 'furry-caiman' - Executing 'predict-f915ca64-0' immediately...
13:32:27.548 | INFO    | Task run 'predict-f915ca64-0' - Finished in state Completed()
13:32:27.574 | INFO    | Flow run 'furry-caiman' - Created task run 'performance-0da4127c-0' for task 'performance'
13:32:27.575 | INFO    | F

13:32:50.501 | INFO    | Flow run 'brainy-lizard' - Executing 'predict-f915ca64-0' immediately...
13:32:50.550 | INFO    | Task run 'predict-f915ca64-0' - Finished in state Completed()
13:32:50.572 | INFO    | Flow run 'brainy-lizard' - Created task run 'performance-0da4127c-0' for task 'performance'
13:32:50.573 | INFO    | Flow run 'brainy-lizard' - Executing 'performance-0da4127c-0' immediately...
13:32:50.622 | INFO    | Task run 'performance-0da4127c-0' - Finished in state Completed()
13:32:50.655 | INFO    | Flow run 'brainy-lizard' - Finished in state Completed()
13:32:51.444 | INFO    | Flow run 'agile-badger' - Created subflow run 'gabby-donkey' for flow 'evaluate-classifier'
13:32:51.500 | INFO    | Flow run 'gabby-donkey' - Created task run 'fit-2157b1fd-0' for task 'fit'
13:32:51.501 | INFO    | Flow run 'gabby-donkey' - Executing 'fit-2157b1fd-0' immediately...
13:32:51.716 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
13:32:51.738 | INFO    | Flow 